## Compiler Optimization

In [ ]:
import torch
import os
import sys
base_dir = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')
sys.path.append(base_dir)
os.environ['TOGSIM_CONFIG']=f"{base_dir}/tutorial/session1/togsim_configs/togsim_config_timing_only.json"

### GeMM + ReLU fusion (Default)

In [ ]:
os.environ['TORCHSIM_DUMP_PATH']=os.path.join(os.getcwd(), "fused")
from Scheduler.scheduler import PyTorchSimRunner
device = PyTorchSimRunner.setup_device().custom_device()

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

def gemm_relu(a, b):
    return torch.relu(torch.matmul(a, b))

opt_fn = torch.compile(dynamic=False)(gemm_relu)
out = opt_fn(input, weight)

In [ ]:
!cat /root/workspace/PyTorchSim/outputs/20251202_060538/togsim_result.log | grep "Total execution cycle"

### Disable fusion

In [ ]:
os.environ['TORCHSIM_DUMP_PATH']=os.path.join(os.getcwd(), "non_fused")
os.environ['TOGSIM_CONFIG']=f"{base_dir}/tutorial/session1/togsim_configs/togsim_config_no_compiler_optimization.json"

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

def gemm_relu(a, b):
    return torch.relu(torch.matmul(a, b))

opt_fn = torch.compile(dynamic=False)(gemm_relu)
out = opt_fn(input, weight)

In [ ]:
!cat /root/workspace/PyTorchSim/outputs/20251202_055530/togsim_result.log | grep "Total execution cycle"
!cat /root/workspace/PyTorchSim/outputs/20251202_055532/togsim_result.log | grep "Total execution cycle"